In [1]:
# Import section
import pandas as pd
import numpy as np
import scipy
import nltk
import spacy
import gensim
import glob
import csv
from spellchecker import SpellChecker
import matplotlib
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import cross_val_score
import sklearn.model_selection
import sklearn.pipeline
import re
from sklearn import svm
from sklearn import *
from sklearn.feature_selection import SelectKBest, VarianceThreshold
from sklearn.feature_selection import chi2
from sklearn.base import BaseEstimator, TransformerMixin
import gensim.models.wrappers.fasttext
from scipy import sparse
from sklearn.model_selection import LeaveOneOut,KFold,train_test_split


# Custom imports
from mr_generic_scripts import *
from mr_cls_SVM import *

In [2]:
# Path to xlsx files folder
path_to_raw_files = 'Data/raw/'


In [3]:
# Get the datasets in dataframes
a_dataset = mr_get_data(path_to_raw_files)
qa_dataset = mr_get_qa_data(path_to_raw_files)

In [4]:
# Sanity check
print(a_dataset[0][1][:5])
print(qa_dataset[0][1][:5])
print(len(a_dataset[0][1]))

   Child_ID                                             Answer  Score  Age  \
0         0   Because they did n't want the woman to find them      2    9   
1         1           Because they Were n't supose to be there      0    9   
2         2                                they have a seccret      0    9   
3         3               because they didnt whant to get cort      1    9   
4         4  Because they did n't want the woman to see them .      1    9   

   Gender           Question  
0       0  SFQuestion_1_Text  
1       1  SFQuestion_1_Text  
2       0  SFQuestion_1_Text  
3       0  SFQuestion_1_Text  
4       1  SFQuestion_1_Text  
   Child_ID                                             Answer  Score  Age  \
0         0  Why did the men hide ? Because they did n't wa...      2    9   
1         1  Why did the men hide ? Because they Were n't s...      0    9   
2         2         Why did the men hide ? they have a seccret      0    9   
3         3  Why did the men hide

In [5]:
# Initialize the spacy model
spacy_mod = spacy.load("en_core_web_sm") 

# Def a funciton for basic preprocessing
# Input is a string
# Output is a sequence of spacy objects
def spacy_tok(response,nlp=spacy_mod):

    # Linguistic preprocessing
    # Ignore missing data, spacy complains
    proc_text = nlp(response) if not pd.isna(response) else ""
    
    # Return the spacy objects
    return proc_text

# Run POS tagging on both input corpora, needed for POS features
a_dataset[-1][1]["Answer"] = a_dataset[-1][1]["Answer"].apply(spacy_tok)
qa_dataset[-1][1]["Answer"] = qa_dataset[-1][1]["Answer"].apply(spacy_tok)

In [6]:
# Create a new instalce of the classifier
# We use the predefined column names, age list from 7 to 14
# C = 0.1, gamma="scale", kernel = "rbf"
svm_cls = MR_SVM(text_cols,[7,8,9,10,11,12,13,14],0.1,'scale','rbf')

In [7]:
# Set evaluation parameters - evaluate by question, evaluate by age; do not return incorrect predictions (experimental)
svm_cls.mr_set_eval_vars(True,True,False)

In [8]:
# Set feature union. No tf-idf. Unigrams and bigrams (tokens), uni-, bi-, tri-grams (chars)
svm_cls.ngram_features(False, (1,2), (1,3))

In [9]:
# 10 fold cross validation, original dataset, without the question
results_1 = svm_cls.mr_kfold_train_test(a_dataset[-1][1],10)

Training
Testing the model on the test set:
Accuracy: 0.68 

Test Macro F1: 0.68 

Evaluating column Question with value SFQuestion_1_Text
Accuracy: 0.7 

Macro F1: 0.48 

Evaluating column Question with value SFQuestion_2_Text
Accuracy: 0.68 

Macro F1: 0.48 

Evaluating column Question with value SFQuestion_3_Text
Accuracy: 0.77 

Macro F1: 0.66 

Evaluating column Question with value SFQuestion_4_Text
Accuracy: 0.61 

Macro F1: 0.43 

Evaluating column Question with value SFQuestion_5_Text
Accuracy: 0.71 

Macro F1: 0.63 

Evaluating column Question with value SFQuestion_6_Text
Accuracy: 0.57 

Macro F1: 0.53 

Evaluating column Question with value SS_Brian_Text
Accuracy: 0.67 

Macro F1: 0.4 

Evaluating column Question with value SS_Peabody_Text
Accuracy: 0.78 

Macro F1: 0.65 

Evaluating column Question with value SS_Prisoner_Text
Accuracy: 0.61 

Macro F1: 0.53 

Evaluating column Question with value SS_Simon_Text
Accuracy: 0.77 

Macro F1: 0.66 

Evaluating column Question wit

C:\Users\Venelin\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Venelin\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Testing the model on the test set:
Accuracy: 0.68 

Test Macro F1: 0.68 

Evaluating column Question with value SFQuestion_1_Text
Accuracy: 0.65 

Macro F1: 0.36 

Evaluating column Question with value SFQuestion_2_Text
Accuracy: 0.62 

Macro F1: 0.47 

Evaluating column Question with value SFQuestion_3_Text
Accuracy: 0.72 

Macro F1: 0.49 

Evaluating column Question with value SFQuestion_4_Text
Accuracy: 0.73 

Macro F1: 0.71 

Evaluating column Question with value SFQuestion_5_Text
Accuracy: 0.77 

Macro F1: 0.7 

Evaluating column Question with value SFQuestion_6_Text
Accuracy: 0.53 

Macro F1: 0.5 

Evaluating column Question with value SS_Brian_Text
Accuracy: 0.78 

Macro F1: 0.49 

Evaluating column Question with value SS_Peabody_Text
Accuracy: 0.78 

Macro F1: 0.61 

Evaluating column Question with value SS_Prisoner_Text
Accuracy: 0.59 

Macro F1: 0.56 

Evaluating column Question with value SS_Simon_Text
Accuracy: 0.74 

Macro F1: 0.61 

Evaluating column Question with value S

C:\Users\Venelin\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Venelin\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Testing the model on the test set:
Accuracy: 0.71 

Test Macro F1: 0.7 

Evaluating column Question with value SFQuestion_1_Text
Accuracy: 0.65 

Macro F1: 0.36 

Evaluating column Question with value SFQuestion_2_Text
Accuracy: 0.62 

Macro F1: 0.46 

Evaluating column Question with value SFQuestion_3_Text
Accuracy: 0.79 

Macro F1: 0.66 

Evaluating column Question with value SFQuestion_4_Text
Accuracy: 0.69 

Macro F1: 0.64 

Evaluating column Question with value SFQuestion_5_Text
Accuracy: 0.75 

Macro F1: 0.7 

Evaluating column Question with value SFQuestion_6_Text
Accuracy: 0.56 

Macro F1: 0.5 

Evaluating column Question with value SS_Brian_Text
Accuracy: 0.85 

Macro F1: 0.68 

Evaluating column Question with value SS_Peabody_Text
Accuracy: 0.82 

Macro F1: 0.69 

Evaluating column Question with value SS_Prisoner_Text
Accuracy: 0.66 

Macro F1: 0.58 

Evaluating column Question with value SS_Simon_Text
Accuracy: 0.87 

Macro F1: 0.8 

Evaluating column Question with value SS_

C:\Users\Venelin\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Testing the model on the test set:
Accuracy: 0.68 

Test Macro F1: 0.68 

Evaluating column Question with value SFQuestion_1_Text
Accuracy: 0.62 

Macro F1: 0.41 

Evaluating column Question with value SFQuestion_2_Text
Accuracy: 0.68 

Macro F1: 0.55 

Evaluating column Question with value SFQuestion_3_Text
Accuracy: 0.65 

Macro F1: 0.46 

Evaluating column Question with value SFQuestion_4_Text
Accuracy: 0.75 

Macro F1: 0.69 

Evaluating column Question with value SFQuestion_5_Text
Accuracy: 0.8 

Macro F1: 0.72 

Evaluating column Question with value SFQuestion_6_Text
Accuracy: 0.58 

Macro F1: 0.54 

Evaluating column Question with value SS_Brian_Text
Accuracy: 0.74 

Macro F1: 0.41 

Evaluating column Question with value SS_Peabody_Text
Accuracy: 0.83 

Macro F1: 0.65 

Evaluating column Question with value SS_Prisoner_Text
Accuracy: 0.53 

Macro F1: 0.48 

Evaluating column Question with value SS_Simon_Text
Accuracy: 0.85 

Macro F1: 0.8 

Evaluating column Question with value S

C:\Users\Venelin\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Testing the model on the test set:
Accuracy: 0.7 

Test Macro F1: 0.69 

Evaluating column Question with value SFQuestion_1_Text
Accuracy: 0.7 

Macro F1: 0.48 

Evaluating column Question with value SFQuestion_2_Text
Accuracy: 0.65 

Macro F1: 0.49 

Evaluating column Question with value SFQuestion_3_Text
Accuracy: 0.73 

Macro F1: 0.5 

Evaluating column Question with value SFQuestion_4_Text
Accuracy: 0.74 

Macro F1: 0.61 

Evaluating column Question with value SFQuestion_5_Text
Accuracy: 0.84 

Macro F1: 0.77 

Evaluating column Question with value SFQuestion_6_Text
Accuracy: 0.5 

Macro F1: 0.45 

Evaluating column Question with value SS_Brian_Text
Accuracy: 0.85 

Macro F1: 0.62 

Evaluating column Question with value SS_Peabody_Text
Accuracy: 0.72 

Macro F1: 0.59 

Evaluating column Question with value SS_Prisoner_Text
Accuracy: 0.57 

Macro F1: 0.5 

Evaluating column Question with value SS_Simon_Text
Accuracy: 0.8 

Macro F1: 0.75 

Evaluating column Question with value SS_Bu

In [11]:
import pprint
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(results_1)

[   (   [   0.68,
            [0.7, 0.68, 0.77, 0.61, 0.71, 0.57, 0.67, 0.78, 0.61, 0.77, 0.62],
            [0.67, 0.65, 0.67, 0.66, 0.71, 0.65, 0.75, 0.5]],
        [   0.68,
            [0.48, 0.48, 0.66, 0.43, 0.63, 0.53, 0.4, 0.65, 0.53, 0.66, 0.57],
            [0.53, 0.64, 0.67, 0.66, 0.71, 0.63, 0.74, 0.33]]),
    (   [   0.68,
            [0.65, 0.62, 0.72, 0.73, 0.77, 0.53, 0.78, 0.78, 0.59, 0.74, 0.61],
            [0.78, 0.75, 0.68, 0.68, 0.65, 0.69, 0.69, 0.0]],
        [   0.68,
            [0.36, 0.47, 0.49, 0.71, 0.7, 0.5, 0.49, 0.61, 0.56, 0.61, 0.61],
            [0.76, 0.73, 0.68, 0.68, 0.64, 0.66, 0.65, 0.0]]),
    (   [   0.71,
            [0.65, 0.62, 0.79, 0.69, 0.75, 0.56, 0.85, 0.82, 0.66, 0.87, 0.53],
            [0.67, 0.67, 0.76, 0.69, 0.71, 0.65, 0.69, 0.67]],
        [   0.7,
            [0.36, 0.46, 0.66, 0.64, 0.7, 0.5, 0.68, 0.69, 0.58, 0.8, 0.51],
            [0.65, 0.66, 0.76, 0.68, 0.7, 0.63, 0.68, 0.56]]),
    (   [   0.68,
            [0.62, 0.68, 

In [12]:
def mr_proc_results(raw_results):
    # Process the results from the 10 runs
    # result format: [acc, acc per q, acc per age], [f1, f1 per q, f1 per age]
    # Ignore ages as they seem to be mostly consistent with global average
    # Ignore accs per question and age as averaging them seems to be consistent with global average
    # Report global acc, global macro f1, average of macro f1 per question
    pr_results = [[acc_score, f1_score,round(sum(qa_s)/11,2),round(sum(qf_s)/11,2)] 
                for ([acc_score, qa_s, aa_s], [f1_score, qf_s, af_s]) in raw_results]
    
    # Throw the list in an np array
    pr_arr = np.array(pr_results)

    # Print the results
    pp = pprint.PrettyPrinter(indent=4)

    print("Acc, F1, Avg-F1-Per-Q")
    pp.pprint(pr_results)
    pp.pprint(np.mean(pr_arr,axis=0))
    
    
    print("\nAcc per question (avg over 10 runs)")
    # Get the result per question
    pr_qa_results = [[qa_s] for ([acc_score, qa_s, aa_s], [f1_score, qf_s, af_s]) in raw_results]
    pr_qa_arr = np.array(pr_qa_results)
    avg_qa = np.mean(pr_qa_arr,axis=0).tolist()
    print(' & '.join([str(round(res,2)) for res in avg_qa[0]]))
    #pp.pprint(np.mean(pr_qa_arr,axis=0))
    
    print("\nF1 per question (avg over 10 runs)")
    pr_qf_results = [[qf_s] for ([acc_score, qa_s, aa_s], [f1_score, qf_s, af_s]) in raw_results]
    pr_qf_arr = np.array(pr_qf_results)
    avg_qf = np.mean(pr_qf_arr,axis=0).tolist()
    print(' & '.join([str(round(res,2)) for res in avg_qf[0]]))
    #pp.pprint(np.mean(pr_qf_arr,axis=0))

In [13]:
mr_proc_results(results_1)

Acc, F1, Avg-F1-Per-Q
[   [0.68, 0.68, 0.68, 0.55],
    [0.68, 0.68, 0.68, 0.56],
    [0.71, 0.7, 0.71, 0.6],
    [0.68, 0.68, 0.68, 0.56],
    [0.69, 0.68, 0.69, 0.54],
    [0.7, 0.69, 0.7, 0.57],
    [0.67, 0.67, 0.67, 0.55],
    [0.69, 0.69, 0.69, 0.58],
    [0.7, 0.7, 0.7, 0.59],
    [0.68, 0.68, 0.68, 0.56]]
array([0.688, 0.685, 0.688, 0.566])

Acc per question (avg over 10 runs)
0.65 & 0.67 & 0.73 & 0.69 & 0.78 & 0.56 & 0.78 & 0.8 & 0.61 & 0.78 & 0.54

F1 per question (avg over 10 runs)
0.4 & 0.49 & 0.57 & 0.63 & 0.71 & 0.5 & 0.51 & 0.65 & 0.54 & 0.69 & 0.51


In [14]:
# 10 fold cross validation, original dataset, including the question
results_2 = svm_cls.mr_kfold_train_test(qa_dataset[-1][1],10)

Training
Testing the model on the test set:
Accuracy: 0.71 

Test Macro F1: 0.71 

Evaluating column Question with value SFQuestion_1_Text


C:\Users\Venelin\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Accuracy: 0.75 

Macro F1: 0.29 

Evaluating column Question with value SFQuestion_2_Text
Accuracy: 0.69 

Macro F1: 0.4 

Evaluating column Question with value SFQuestion_3_Text
Accuracy: 0.71 

Macro F1: 0.49 

Evaluating column Question with value SFQuestion_4_Text
Accuracy: 0.64 

Macro F1: 0.43 

Evaluating column Question with value SFQuestion_5_Text
Accuracy: 0.72 

Macro F1: 0.51 

Evaluating column Question with value SFQuestion_6_Text
Accuracy: 0.62 

Macro F1: 0.42 

Evaluating column Question with value SS_Brian_Text
Accuracy: 0.79 

Macro F1: 0.38 

Evaluating column Question with value SS_Peabody_Text
Accuracy: 0.75 

Macro F1: 0.51 

Evaluating column Question with value SS_Prisoner_Text
Accuracy: 0.6 

Macro F1: 0.52 

Evaluating column Question with value SS_Simon_Text
Accuracy: 0.75 

Macro F1: 0.57 

Evaluating column Question with value SS_Burglar_Text
Accuracy: 0.78 

Macro F1: 0.71 

Evaluating column Age with value 7
Accuracy: 0.67 

Macro F1: 0.47 

Evaluating c

C:\Users\Venelin\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Accuracy: 0.74 

Macro F1: 0.28 

Evaluating column Question with value SFQuestion_2_Text
Accuracy: 0.65 

Macro F1: 0.36 

Evaluating column Question with value SFQuestion_3_Text
Accuracy: 0.7 

Macro F1: 0.46 

Evaluating column Question with value SFQuestion_4_Text
Accuracy: 0.74 

Macro F1: 0.63 

Evaluating column Question with value SFQuestion_5_Text
Accuracy: 0.67 

Macro F1: 0.48 

Evaluating column Question with value SFQuestion_6_Text
Accuracy: 0.52 

Macro F1: 0.37 

Evaluating column Question with value SS_Brian_Text
Accuracy: 0.81 

Macro F1: 0.35 

Evaluating column Question with value SS_Peabody_Text
Accuracy: 0.84 

Macro F1: 0.56 

Evaluating column Question with value SS_Prisoner_Text
Accuracy: 0.58 

Macro F1: 0.53 

Evaluating column Question with value SS_Simon_Text
Accuracy: 0.73 

Macro F1: 0.57 

Evaluating column Question with value SS_Burglar_Text
Accuracy: 0.68 

Macro F1: 0.67 

Evaluating column Age with value 7
Accuracy: 1.0 

Macro F1: 1.0 

Evaluating co

...

Accuracy: 0.7 

Macro F1: 0.36 

Evaluating column Question with value SFQuestion_2_Text
Accuracy: 0.59 

Macro F1: 0.31 

Evaluating column Question with value SFQuestion_3_Text
Accuracy: 0.78 

Macro F1: 0.54 

Evaluating column Question with value SFQuestion_4_Text
Accuracy: 0.71 

Macro F1: 0.65 

Evaluating column Question with value SFQuestion_5_Text
Accuracy: 0.7 

Macro F1: 0.49 

Evaluating column Question with value SFQuestion_6_Text
Accuracy: 0.61 

Macro F1: 0.49 

Evaluating column Question with value SS_Brian_Text
Accuracy: 0.79 

Macro F1: 0.3 

Evaluating column Question with value SS_Peabody_Text
Accuracy: 0.87 

Macro F1: 0.8 

Evaluating column Question with value SS_Prisoner_Text
Accuracy: 0.61 

Macro F1: 0.56 

Evaluating column Question with value SS_Simon_Text
Accuracy: 0.75 

Macro F1: 0.53 

Evaluating column Question with value SS_Burglar_Text
Accuracy: 0.74 

Macro F1: 0.66 

Evaluating column Age with value 7
Accuracy: 0.67 

Macro F1: 0.58 

Evaluating col

C:\Users\Venelin\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Accuracy: 0.64 

Macro F1: 0.44 

Evaluating column Question with value SFQuestion_2_Text
Accuracy: 0.63 

Macro F1: 0.37 

Evaluating column Question with value SFQuestion_3_Text
Accuracy: 0.78 

Macro F1: 0.55 

Evaluating column Question with value SFQuestion_4_Text
Accuracy: 0.66 

Macro F1: 0.58 

Evaluating column Question with value SFQuestion_5_Text
Accuracy: 0.74 

Macro F1: 0.51 

Evaluating column Question with value SFQuestion_6_Text
Accuracy: 0.57 

Macro F1: 0.42 

Evaluating column Question with value SS_Brian_Text
Accuracy: 0.89 

Macro F1: 0.49 

Evaluating column Question with value SS_Peabody_Text
Accuracy: 0.79 

Macro F1: 0.62 

Evaluating column Question with value SS_Prisoner_Text
Accuracy: 0.64 

Macro F1: 0.56 

Evaluating column Question with value SS_Simon_Text
Accuracy: 0.77 

Macro F1: 0.51 

Evaluating column Question with value SS_Burglar_Text
Accuracy: 0.75 

Macro F1: 0.7 

Evaluating column Age with value 7
Accuracy: 0.62 

Macro F1: 0.61 

Evaluating 

C:\Users\Venelin\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Accuracy: 0.76 

Macro F1: 0.36 

Evaluating column Question with value SFQuestion_2_Text
Accuracy: 0.64 

Macro F1: 0.4 

Evaluating column Question with value SFQuestion_3_Text
Accuracy: 0.62 

Macro F1: 0.38 

Evaluating column Question with value SFQuestion_4_Text
Accuracy: 0.64 

Macro F1: 0.46 

Evaluating column Question with value SFQuestion_5_Text
Accuracy: 0.69 

Macro F1: 0.48 

Evaluating column Question with value SFQuestion_6_Text
Accuracy: 0.61 

Macro F1: 0.43 

Evaluating column Question with value SS_Brian_Text
Accuracy: 0.79 

Macro F1: 0.39 

Evaluating column Question with value SS_Peabody_Text
Accuracy: 0.81 

Macro F1: 0.71 

Evaluating column Question with value SS_Prisoner_Text
Accuracy: 0.67 

Macro F1: 0.57 

Evaluating column Question with value SS_Simon_Text
Accuracy: 0.7 

Macro F1: 0.5 

Evaluating column Question with value SS_Burglar_Text
Accuracy: 0.77 

Macro F1: 0.71 

Evaluating column Age with value 7
Accuracy: 0.57 

Macro F1: 0.52 

Evaluating co

In [15]:
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(results_2)

[   (   [   0.71,
            [0.75, 0.69, 0.71, 0.64, 0.72, 0.62, 0.79, 0.75, 0.6, 0.75, 0.78],
            [0.67, 0.65, 0.71, 0.68, 0.76, 0.66, 0.77, 0.5]],
        [   0.71,
            [0.29, 0.4, 0.49, 0.43, 0.51, 0.42, 0.38, 0.51, 0.52, 0.57, 0.71],
            [0.47, 0.64, 0.71, 0.68, 0.76, 0.65, 0.75, 0.33]]),
    (   [   0.69,
            [0.74, 0.65, 0.7, 0.74, 0.67, 0.52, 0.81, 0.84, 0.58, 0.73, 0.68],
            [1.0, 0.78, 0.68, 0.7, 0.68, 0.69, 0.69, 0.0]],
        [   0.69,
            [0.28, 0.36, 0.46, 0.63, 0.48, 0.37, 0.35, 0.56, 0.53, 0.57, 0.67],
            [1.0, 0.78, 0.69, 0.7, 0.67, 0.66, 0.64, 0.0]]),
    (   [   0.72,
            [0.72, 0.62, 0.77, 0.64, 0.69, 0.62, 0.84, 0.84, 0.67, 0.75, 0.74],
            [0.83, 0.7, 0.76, 0.73, 0.69, 0.64, 0.7, 0.67]],
        [   0.71,
            [0.34, 0.35, 0.51, 0.44, 0.51, 0.42, 0.36, 0.57, 0.6, 0.53, 0.69],
            [0.8, 0.69, 0.76, 0.73, 0.69, 0.63, 0.68, 0.56]]),
    (   [   0.69,
            [0.69, 0.69, 0.

In [16]:
mr_proc_results(results_2)

Acc, F1, Avg-F1-Per-Q
[   [0.71, 0.71, 0.71, 0.48],
    [0.69, 0.69, 0.7, 0.48],
    [0.72, 0.71, 0.72, 0.48],
    [0.69, 0.69, 0.69, 0.47],
    [0.73, 0.72, 0.73, 0.48],
    [0.72, 0.71, 0.71, 0.51],
    [0.7, 0.7, 0.7, 0.51],
    [0.71, 0.71, 0.71, 0.52],
    [0.71, 0.71, 0.71, 0.52],
    [0.7, 0.7, 0.7, 0.49]]
array([0.708, 0.705, 0.708, 0.494])

Acc per question (avg over 10 runs)
0.71 & 0.65 & 0.7 & 0.68 & 0.72 & 0.61 & 0.81 & 0.81 & 0.63 & 0.73 & 0.74

F1 per question (avg over 10 runs)
0.34 & 0.38 & 0.47 & 0.54 & 0.51 & 0.43 & 0.37 & 0.62 & 0.55 & 0.53 & 0.68
